In [1]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import f1_score, accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.inspection import permutation_importance
from imblearn.over_sampling import SMOTE
import mplfinance as mpf
import matplotlib.dates as mdates
import matplotlib.ticker as mticker
from mplfinance.original_flavor import candlestick_ohlc
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error
import seaborn as sns
from finta import TA
import backtrader as bt

In [2]:
# read into df
df = pd.read_csv(r"C:\Users\ganes\OneDrive\Desktop\capestone\ETH_training_data_2023-01-01_to_2023-10-31.csv")


In [3]:
import talib
import pandas as pd

# Calculate SMAs using TA-Lib
df['SMA_2'] = talib.SMA(df['close'], timeperiod=2)    # 2-period SMA
df['SMA_5'] = talib.SMA(df['close'], timeperiod=5)    # 5-period SMA
df['SMA_10'] = talib.SMA(df['close'], timeperiod=10)  # 10-period SMA
df['SMA_15'] = talib.SMA(df['close'], timeperiod=15)  # 15-period SMA
df['SMA_20'] = talib.SMA(df['close'], timeperiod=20)  # 20-period SMA
df['SMA_50'] = talib.SMA(df['close'], timeperiod=50)  # 50-period SMA

# Calculate SMAs using TA-Lib
df['EMA_2'] = talib.EMA(df['close'], timeperiod=2)    # 2-period EMA
df['EMA_5'] = talib.EMA(df['close'], timeperiod=5)    # 5-period EMA
df['EMA_10'] = talib.EMA(df['close'], timeperiod=10)  # 10-period EMA
df['EMA_15'] = talib.EMA(df['close'], timeperiod=15)  # 15-period EMA
df['EMA_20'] = talib.EMA(df['close'], timeperiod=20)  # 20-period EMA
df['EMA_50'] = talib.EMA(df['close'], timeperiod=50)  # 50-period EMA

# Display the DataFrame with the moving averages
df.head()


,unix,date,symbol,open,high,low,close,volume,volume_from,marketorder_volume,...,SMA_10,SMA_15,SMA_20,SMA_50,EMA_2,EMA_5,EMA_10,EMA_15,EMA_20,EMA_50
0,1672531200000,2023-01-01 00:00:00,ETHUSDT,1196.13,1196.14,1195.92,1196.13,65.5003,78336.933825,20.5552,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1672531260000,2023-01-01 00:01:00,ETHUSDT,1196.13,1196.25,1196.08,1196.09,61.1703,73172.254498,6.0606,...,NaN,NaN,NaN,NaN,1196.110000,NaN,NaN,NaN,NaN,NaN
2,1672531320000,2023-01-01 00:02:00,ETHUSDT,1196.09,1196.10,1195.85,1195.85,32.9332,39389.839241,8.7272,...,NaN,NaN,NaN,NaN,1195.936667,NaN,NaN,NaN,NaN,NaN
3,1672531380000,2023-01-01 00:03:00,ETHUSDT,1195.85,1195.86,1195.82,1195.82,56.8456,67978.442332,5.7935,...,NaN,NaN,NaN,NaN,1195.858889,NaN,NaN,NaN,NaN,NaN
4,1672531440000,2023-01-01 00:04:00,ETHUSDT,1195.82,1196.32,1195.65,1196.32,118.1098,141237.845266,82.1776,...,NaN,NaN,NaN,NaN,1196.166296,1196.042,NaN,NaN,NaN,NaN


In [4]:
df['date'] = pd.to_datetime(df['date'])

# Set 'datetime' as the index
df.set_index('date', inplace=True)
df.head()

,unix,symbol,open,high,low,close,volume,volume_from,marketorder_volume,marketorder_volume_from,...,SMA_10,SMA_15,SMA_20,SMA_50,EMA_2,EMA_5,EMA_10,EMA_15,EMA_20,EMA_50
date,,,,,,,,,,,,,,,,,,,,,
2023-01-01 00:00:00,1672531200000,ETHUSDT,1196.13,1196.14,1195.92,1196.13,65.5003,78336.933825,20.5552,24584.544571,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-01 00:01:00,1672531260000,ETHUSDT,1196.13,1196.25,1196.08,1196.09,61.1703,73172.254498,6.0606,7249.354624,...,NaN,NaN,NaN,NaN,1196.110000,NaN,NaN,NaN,NaN,NaN
2023-01-01 00:02:00,1672531320000,ETHUSDT,1196.09,1196.10,1195.85,1195.85,32.9332,39389.839241,8.7272,10438.083144,...,NaN,NaN,NaN,NaN,1195.936667,NaN,NaN,NaN,NaN,NaN
2023-01-01 00:03:00,1672531380000,ETHUSDT,1195.85,1195.86,1195.82,1195.82,56.8456,67978.442332,5.7935,6928.150698,...,NaN,NaN,NaN,NaN,1195.858889,NaN,NaN,NaN,NaN,NaN
2023-01-01 00:04:00,1672531440000,ETHUSDT,1195.82,1196.32,1195.65,1196.32,118.1098,141237.845266,82.1776,98271.860291,...,NaN,NaN,NaN,NaN,1196.166296,1196.042,NaN,NaN,NaN,NaN


In [5]:
# Creating RSI for different time periods

# Calculate RSI for scalping (7-period)
df['RSI_7'] = talib.RSI(df['close'], timeperiod=7)

# Calculate RSI for day trading (14-period)
df['RSI_14'] = talib.RSI(df['close'], timeperiod=14)

# Calculate RSI for swing trading (21-period)
df['RSI_21'] = talib.RSI(df['close'], timeperiod=21)

# Display the DataFrame with the new RSI columns
df[['close', 'RSI_7', 'RSI_14', 'RSI_21']].head(n=10)


,close,RSI_7,RSI_14,RSI_21
date,,,,
2023-01-01 00:00:00,1196.13,NaN,NaN,NaN
2023-01-01 00:01:00,1196.09,NaN,NaN,NaN
2023-01-01 00:02:00,1195.85,NaN,NaN,NaN
2023-01-01 00:03:00,1195.82,NaN,NaN,NaN
2023-01-01 00:04:00,1196.32,NaN,NaN,NaN
2023-01-01 00:05:00,1196.60,NaN,NaN,NaN
2023-01-01 00:06:00,1196.51,NaN,NaN,NaN
2023-01-01 00:07:00,1196.39,60.000000,NaN,NaN
2023-01-01 00:08:00,1195.61,35.294118,NaN,NaN


In [6]:
# Calculate MACD with adjusted periods (e.g., 7, 21, 5)
df['MACD'], df['MACD_Signal'], df['MACD_Hist'] = talib.MACD(df['close'], fastperiod=7, slowperiod=21, signalperiod=5)

# Display the DataFrame with MACD, Signal Line, and Histogram columns
df[['close', 'MACD', 'MACD_Signal', 'MACD_Hist']].tail()

# Bollinger Bands with 10 periods and 1.5 standard deviations
df['upper_band_10_1_5'], df['middle_band_10_1_5'], df['lower_band_10_1_5'] = talib.BBANDS(
    df['close'], timeperiod=10, nbdevup=1.5, nbdevdn=1.5, matype=1)

# Bollinger Bands with 10 periods and 2 standard deviations
df['upper_band_10_2'], df['middle_band_10_2'], df['lower_band_10_2'] = talib.BBANDS(
    df['close'], timeperiod=10, nbdevup=2, nbdevdn=2, matype=1)

# Bollinger Bands with 20 periods and 1.5 standard deviations
df['upper_band_20_1_5'], df['middle_band_20_1_5'], df['lower_band_20_1_5'] = talib.BBANDS(
    df['close'], timeperiod=20, nbdevup=1.5, nbdevdn=1.5, matype=1)

# Bollinger Bands with 20 periods and 2 standard deviations
df['upper_band_20_2'], df['middle_band_20_2'], df['lower_band_20_2'] = talib.BBANDS(
    df['close'], timeperiod=20, nbdevup=2, nbdevdn=2, matype=1)


In [7]:
max_close = df['close'].max()
target_max_price = 100
scale_factor = max_close / target_max_price

df['open'] /= scale_factor
df['high'] /= scale_factor
df['low'] /= scale_factor
df['close'] /= scale_factor

In [8]:
# Splitting the data into train and test sets
df_size = int(0.5 * len(df))
df = df.iloc[:df_size] 
#df = df.iloc[df_size:]

In [9]:
# # Splitting the data into train and test sets
# train_size = int(0.8 * len(df))
# df_train = df.iloc[:train_size]
# df_test = df.iloc[train_size:]

In [10]:
#paste the extra cide here

In [11]:
# Splitting the data into train and test sets
df_size = int(0.8 * len(df))
# df = df.iloc[:df_size]   #train
df = df.iloc[df_size:]  #test

In [12]:
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning, message="divide by zero encountered in scalar divide")


In [13]:
import pandas as pd
import numpy as np
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from dateutil import parser

# Renaming columns to fit the Backtesting.py requirements
df = df.rename(columns={
    'open': 'Open',
    'high': 'High',
    'low': 'Low',
    'close': 'Close',
    'volume': 'Volume'  
})

# Function to fetch data based on time frame
def fetch_data_for_time_frame(df, start_date, time_frame):
    if 'min' in time_frame:
        minutes = int(time_frame.replace('min', ''))
        end_date = start_date + pd.Timedelta(minutes=minutes)
    elif 'hr' in time_frame:
        hours = int(time_frame.replace('hr', ''))
        end_date = start_date + pd.Timedelta(hours=hours)
    elif 'week' in time_frame:
        weeks = int(time_frame.replace(' week', ''))
        end_date = start_date + pd.Timedelta(weeks=weeks)
    elif 'day' in time_frame or '24hr' in time_frame:
        days = 1  # Assuming 1 day for 24hr
        end_date = start_date + pd.Timedelta(days=days)
    else:
        print("Unsupported time frame. Please enter a valid time frame (e.g., '15min', '1hr', '4hr', '8hr', '24hr', or '1 week').")
        return None, None

    mask = (df.index >= start_date) & (df.index < end_date)
    result_df = df.loc[mask]
    return result_df, end_date


# Define the strategy classes (SMA, RSI, MACD, Bollinger Bands, etc.)
# 1. SMA 2,5 Strategy
class SMA25Strategy(Strategy):
    def init(self):
        self.sma2 = self.I(lambda x: x, self.data.SMA_2)
        self.sma5 = self.I(lambda x: x, self.data.SMA_5)

    def next(self):
        if crossover(self.sma2, self.sma5):
            if not self.position:
                self.buy()
        elif crossover(self.sma5, self.sma2):
            if self.position:
                self.position.close()

#2. SMA 5,10 Strategy
class SMA510Strategy(Strategy):
    def init(self):
        self.sma5 = self.I(lambda x: x, self.data.SMA_5)
        self.sma10 = self.I(lambda x: x, self.data.SMA_10)

    def next(self):
        if crossover(self.sma5, self.sma10):
            if not self.position:
                self.buy()
        elif crossover(self.sma10, self.sma5):
            if self.position:
                self.position.close()


#3. SMA 10,15 Strategy
class SMA1015Strategy(Strategy):
    def init(self):
        self.sma10 = self.I(lambda x: x, self.data.SMA_10)
        self.sma15 = self.I(lambda x: x, self.data.SMA_15)

    def next(self):
        if crossover(self.sma10, self.sma15):
            if not self.position:
                self.buy()
        elif crossover(self.sma15, self.sma10):
            if self.position:
                self.position.close()

#4. SMA 15,20 Strategy
class SMA1520Strategy(Strategy):
    def init(self):
        self.sma15 = self.I(lambda x: x, self.data.SMA_15)
        self.sma20 = self.I(lambda x: x, self.data.SMA_20)

    def next(self):
        if crossover(self.sma15, self.sma20):
            if not self.position:
                self.buy()
        elif crossover(self.sma20, self.sma15):
            if self.position:
                self.position.close()

#5. SMA 20,50 Strategy
class SMA2050Strategy(Strategy):
    def init(self):
        self.sma20 = self.I(lambda x: x, self.data.SMA_20)
        self.sma50 = self.I(lambda x: x, self.data.SMA_50)

    def next(self):
        if crossover(self.sma20, self.sma50):
            if not self.position:
                self.buy()
        elif crossover(self.sma50, self.sma20):
            if self.position:
                self.position.close()

#6. EMA 2,5 Strategy
class EMA25Strategy(Strategy):
    def init(self):
        self.ema2 = self.I(lambda x: x, self.data.EMA_2)
        self.ema5 = self.I(lambda x: x, self.data.EMA_5)

    def next(self):
        if crossover(self.ema2, self.ema5):
            if not self.position:
                self.buy()
        elif crossover(self.ema5, self.ema2):
            if self.position:
                self.position.close()

#7. EMA 5,10 Strategy
class EMA510Strategy(Strategy):
    def init(self):
        self.ema5 = self.I(lambda x: x, self.data.EMA_5)
        self.ema10 = self.I(lambda x: x, self.data.EMA_10)

    def next(self):
        if crossover(self.ema5, self.ema10):
            if not self.position:
                self.buy()
        elif crossover(self.ema10, self.ema5):
            if self.position:
                self.position.close()

#8. EMA 10,15 Strategy
class EMA1015Strategy(Strategy):
    def init(self):
        self.ema10 = self.I(lambda x: x, self.data.EMA_10)
        self.ema15 = self.I(lambda x: x, self.data.EMA_15)

    def next(self):
        if crossover(self.ema10, self.ema15):
            if not self.position:
                self.buy()
        elif crossover(self.ema15, self.ema10):
            if self.position:
                self.position.close()

#9. EMA 15,20 Strategy
class EMA1520Strategy(Strategy):
    def init(self):
        self.ema15 = self.I(lambda x: x, self.data.EMA_15)
        self.ema20 = self.I(lambda x: x, self.data.EMA_20)

    def next(self):
        if crossover(self.ema15, self.ema20):
            if not self.position:
                self.buy()
        elif crossover(self.ema20, self.ema15):
            if self.position:
                self.position.close()

#10. EMA 20,50 Strategy
class EMA2050Strategy(Strategy):
    def init(self):
        self.ema20 = self.I(lambda x: x, self.data.EMA_20)
        self.ema50 = self.I(lambda x: x, self.data.EMA_50)

    def next(self):
        if crossover(self.ema20, self.ema50):
            if not self.position:
                self.buy()
        elif crossover(self.ema50, self.ema20):
            if self.position:
                self.position.close()

#RSI Strategy Definitions
#1. RSI 7 with 30/70 Strategy
class RSI7_30_70_Strategy(Strategy):
    def init(self):
        self.rsi7 = self.I(lambda x: x, self.data.RSI_7)

    def next(self):
        if self.rsi7[-1] < 30:
            if not self.position:
                self.buy()
        elif self.rsi7[-1] > 70:
            if self.position:
                self.position.close()

#2. RSI 7 with 25/75 Strategy
class RSI7_25_75_Strategy(Strategy):
    def init(self):
        self.rsi7 = self.I(lambda x: x, self.data.RSI_7)

    def next(self):
        if self.rsi7[-1] < 25:
            if not self.position:
                self.buy()
        elif self.rsi7[-1] > 75:
            if self.position:
                self.position.close()

#3. RSI 7 with 35/65 Strategy
class RSI7_35_65_Strategy(Strategy):
    def init(self):
        self.rsi7 = self.I(lambda x: x, self.data.RSI_7)

    def next(self):
        if self.rsi7[-1] < 35:
            if not self.position:
                self.buy()
        elif self.rsi7[-1] > 65:
            if self.position:
                self.position.close()

#4. RSI 7 with 40/60 Strategy
class RSI7_40_60_Strategy(Strategy):
    def init(self):
        self.rsi7 = self.I(lambda x: x, self.data.RSI_7)

    def next(self):
        if self.rsi7[-1] < 40:
            if not self.position:
                self.buy()
        elif self.rsi7[-1] > 60:
            if self.position:
                self.position.close()

#5. RSI 14 with 30/70 Strategy
class RSI14_30_70_Strategy(Strategy):
    def init(self):
        self.rsi14 = self.I(lambda x: x, self.data.RSI_14)

    def next(self):
        if self.rsi14[-1] < 30:
            if not self.position:
                self.buy()
        elif self.rsi14[-1] > 70:
            if self.position:
                self.position.close()

#6. RSI 14 with 25/75 Strategy
class RSI14_25_75_Strategy(Strategy):
    def init(self):
        self.rsi14 = self.I(lambda x: x, self.data.RSI_14)

    def next(self):
        if self.rsi14[-1] < 25:
            if not self.position:
                self.buy()
        elif self.rsi14[-1] > 75:
            if self.position:
                self.position.close()

#7. RSI 14 with 35/65 Strategy
class RSI14_35_65_Strategy(Strategy):
    def init(self):
        self.rsi14 = self.I(lambda x: x, self.data.RSI_14)

    def next(self):
        if self.rsi14[-1] < 35:
            if not self.position:
                self.buy()
        elif self.rsi14[-1] > 65:
            if self.position:
                self.position.close()

#8. RSI 14 with 40/60 Strategy
class RSI14_40_60_Strategy(Strategy):
    def init(self):
        self.rsi14 = self.I(lambda x: x, self.data.RSI_14)

    def next(self):
        if self.rsi14[-1] < 40:
            if not self.position:
                self.buy()
        elif self.rsi14[-1] > 60:
            if self.position:
                self.position.close()

#9. RSI 21 with 30/70 Strategy
class RSI21_30_70_Strategy(Strategy):
    def init(self):
        self.rsi21 = self.I(lambda x: x, self.data.RSI_21)

    def next(self):
        if self.rsi21[-1] < 30:
            if not self.position:
                self.buy()
        elif self.rsi21[-1] > 70:
            if self.position:
                self.position.close()

#10. RSI 21 with 25/75 Strategy
class RSI21_25_75_Strategy(Strategy):
    def init(self):
        self.rsi21 = self.I(lambda x: x, self.data.RSI_21)

    def next(self):
        if self.rsi21[-1] < 25:
            if not self.position:
                self.buy()
        elif self.rsi21[-1] > 75:
            if self.position:
                self.position.close()

#11. RSI 21 with 35/65 Strategy
class RSI21_35_65_Strategy(Strategy):
    def init(self):
        self.rsi21 = self.I(lambda x: x, self.data.RSI_21)

    def next(self):
        if self.rsi21[-1] < 35:
            if not self.position:
                self.buy()
        elif self.rsi21[-1] > 65:
            if self.position:
                self.position.close()

#2. RSI 21 with 40/60 Strategy
class RSI21_40_60_Strategy(Strategy):
    def init(self):
        self.rsi21 = self.I(lambda x: x, self.data.RSI_21)

    def next(self):
        if self.rsi21[-1] < 40:
            if not self.position:
                self.buy()
        elif self.rsi21[-1] > 60:
            if self.position:
                self.position.close()

#MACD Strategy Definitions
#1. MACD (7,21,5) Strategy
class MACD7_21_5_Strategy(Strategy):
    def init(self):
        self.macd = self.I(lambda x: x, self.data.MACD)
        self.macd_signal = self.I(lambda x: x, self.data.MACD_Signal)

    def next(self):
        if crossover(self.macd, self.macd_signal):
            if not self.position:
                self.buy()
        elif crossover(self.macd_signal, self.macd):
            if self.position:
                self.position.close()


#Bollinger Bands Strategy Definitions
#1. Bollinger Bands (10, 1.5) Strategy
class BollingerBands10_1_5_Strategy(Strategy):
    def init(self):
        self.upper_band = self.I(lambda x: x, self.data.upper_band_10_1_5)
        self.lower_band = self.I(lambda x: x, self.data.lower_band_10_1_5)
        self.middle_band = self.I(lambda x: x, self.data.middle_band_10_1_5)

    def next(self):
        if self.data.Close[-1] < self.lower_band[-1]:
            if not self.position:
                self.buy()
        elif self.data.Close[-1] > self.upper_band[-1]:
            if self.position:
                self.position.close()

#2. Bollinger Bands (10, 2) Strategy
class BollingerBands10_2_Strategy(Strategy):
    def init(self):
        self.upper_band = self.I(lambda x: x, self.data.upper_band_10_2)
        self.lower_band = self.I(lambda x: x, self.data.lower_band_10_2)
        self.middle_band = self.I(lambda x: x, self.data.middle_band_10_2)

    def next(self):
        if self.data.Close[-1] < self.lower_band[-1]:
            if not self.position:
                self.buy()
        elif self.data.Close[-1] > self.upper_band[-1]:
            if self.position:
                self.position.close()

#3. Bollinger Bands (20, 1.5) Strategy
class BollingerBands20_1_5_Strategy(Strategy):
    def init(self):
        self.upper_band = self.I(lambda x: x, self.data.upper_band_20_1_5)
        self.lower_band = self.I(lambda x: x, self.data.lower_band_20_1_5)
        self.middle_band = self.I(lambda x: x, self.data.middle_band_20_1_5)

    def next(self):
        if self.data.Close[-1] < self.lower_band[-1]:
            if not self.position:
                self.buy()
        elif self.data.Close[-1] > self.upper_band[-1]:
            if self.position:
                self.position.close()

#4. Bollinger Bands (20, 2) Strategy
class BollingerBands20_2_Strategy(Strategy):
    def init(self):
        self.upper_band = self.I(lambda x: x, self.data.upper_band_20_2)
        self.lower_band = self.I(lambda x: x, self.data.lower_band_20_2)
        self.middle_band = self.I(lambda x: x, self.data.middle_band_20_2)

    def next(self):
        if self.data.Close[-1] < self.lower_band[-1]:
            if not self.position:
                self.buy()
        elif self.data.Close[-1] > self.upper_band[-1]:
            if self.position:
                self.position.close()

#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#


# ... All your strategies here ...
strategies = {
    # SMA Strategies
    'SMA_2_5': SMA25Strategy,
    'SMA_5_10': SMA510Strategy,
    'SMA_10_15': SMA1015Strategy,
    'SMA_15_20': SMA1520Strategy,
    'SMA_20_50': SMA2050Strategy,

    # EMA Strategies
    'EMA_2_5': EMA25Strategy,
    'EMA_5_10': EMA510Strategy,
    'EMA_10_15': EMA1015Strategy,
    'EMA_15_20': EMA1520Strategy,
    'EMA_20_50': EMA2050Strategy,

    # RSI Strategies (30/70, 25/75, 35/65, 40/60)
    'RSI_7_30_70': RSI7_30_70_Strategy,
    'RSI_7_25_75': RSI7_25_75_Strategy,
    'RSI_7_35_65': RSI7_35_65_Strategy,
    'RSI_7_40_60': RSI7_40_60_Strategy,
    'RSI_14_30_70': RSI14_30_70_Strategy,
    'RSI_14_25_75': RSI14_25_75_Strategy,
    'RSI_14_35_65': RSI14_35_65_Strategy,
    'RSI_14_40_60': RSI14_40_60_Strategy,
    'RSI_21_30_70': RSI21_30_70_Strategy,
    'RSI_21_25_75': RSI21_25_75_Strategy,
    'RSI_21_35_65': RSI21_35_65_Strategy,
    'RSI_21_40_60': RSI21_40_60_Strategy,

    # MACD Strategies (Different Periods)
    'MACD_7_21_5': MACD7_21_5_Strategy,

    # Bollinger Bands Strategies (Different Periods and Standard Deviations)
    'BB_10_1.5': BollingerBands10_1_5_Strategy,
    'BB_10_2': BollingerBands10_2_Strategy,
    'BB_20_1.5': BollingerBands20_1_5_Strategy,
    'BB_20_2': BollingerBands20_2_Strategy
}


# Function to calculate Sharpe Ratio, Annualized Volatility, and Annualized Return
def calculate_metrics(results, periods_per_year, market_return):
    metrics = []  # To store results for each timeframe

    for start, end, output in results:
        # Extracting the equity curve
        equity_curve = output['_equity_curve']

        # Calculate cumulative return
        starting_equity = equity_curve['Equity'].iloc[0]
        ending_equity = equity_curve['Equity'].iloc[-1]
        cumulative_return = (ending_equity - starting_equity) / starting_equity

        # Calculate annualized return
        total_periods = len(equity_curve)
        annualized_return = (1 + cumulative_return) ** (periods_per_year / total_periods) - 1

        # Use the previously defined function to calculate annualized volatility
        annualized_volatility = calculate_volatility(equity_curve, periods_per_year)

        # Calculate daily returns and standard deviation for Sharpe Ratio
        equity_curve['Return'] = equity_curve['Equity'].pct_change()
        returns = equity_curve['Return'].dropna()  # Drop NaN values

        # Calculate Sharpe Ratio (annualized)
        sharpe_ratio = annualized_return / annualized_volatility if annualized_volatility != 0 else 0

        # Strategy vs Market Return Ratio
        strategy_vs_market_return = cumulative_return / market_return if market_return != 0 else np.nan

        # Saving results
        metrics.append({
            'Start': start,
            'End': end,
            'Sharpe Ratio': sharpe_ratio,
            'Annualized Volatility': annualized_volatility,
            'Annualized Return': annualized_return,
            'Equity Final [$]': output['Equity Final [$]'],
            '# Trades': output['# Trades'],
            'Strategy vs Market Return Ratio': strategy_vs_market_return,  # Added
            'Market Return': market_return  # Adding market return to metrics
        })

    return pd.DataFrame(metrics)


# Function to calculate maximum drawdown
def calculate_max_drawdown(equity_curve):
    running_max = equity_curve.cummax()
    drawdown = (equity_curve - running_max) / running_max
    max_drawdown = drawdown.min() * 100  # Converting to percentage
    return abs(max_drawdown)

# Function to run and save backtest
def run_and_save_backtest(strategy_class, df_backtest):
    bt = Backtest(df_backtest, strategy_class, cash=10000, commission=.002, exclusive_orders=True)
    output = bt.run()
    # Calculate maximum drawdown
    equity_curve = output['_equity_curve']['Equity']
    max_drawdown = calculate_max_drawdown(equity_curve)
    # Check for maximum drawdown condition
    if max_drawdown > 20:
        # print(f"Max drawdown exceeded 20% for {strategy_class.__name__}. Stopping strategy.")
        return None
    output['Max Drawdown [%]'] = max_drawdown
    return output

def calculate_volatility(equity_curve, periods_per_year):
    # Calculate returns (daily)
    equity_curve['Return'] = equity_curve['Equity'].pct_change()
    returns = equity_curve['Return'].dropna()

    # Calculate daily volatility (sigma_day) for each trading day
    daily_volatility = returns.rolling(window=30).std()  # 30-day rolling standard deviation

    # Average daily volatility over the last 30 days
    avg_daily_volatility = daily_volatility.mean()

    # Annualize volatility (scaling by the square root of the number of trading days in a year)
    annualized_volatility = avg_daily_volatility * np.sqrt(periods_per_year)

    return annualized_volatility


# Main execution
# Main execution
if __name__ == "__main__":
    time_frame = input("Enter the desired time frame (e.g., '15min', '1hr', '4hr', '8hr', '24hr', '1 week'): ").strip()
    
    time_frame_mapping = {
        '15min': 525600 / 15,
        '1hr': 8760,
        '4hr': 8760 / 4,
        '8hr': 8760 / 8,
        '24hr': 365,
        '1 week': 52
    }

    if time_frame not in time_frame_mapping:
        print(f"Unsupported timeframe: {time_frame}")
        exit()

    periods_per_year = time_frame_mapping[time_frame]

    # Define market return (cumulative return from Buy and Hold strategy)
    market_return = (df['Close'].iloc[-1] - df['Close'].iloc[0]) / df['Close'].iloc[0]

    results = []
    start_date = df.index.min()

    while start_date < df.index.max():
        filtered_data, end_date = fetch_data_for_time_frame(df, start_date, time_frame)
        if filtered_data is None or filtered_data.empty:
            break
        df_backtest = filtered_data.loc[filtered_data['EMA_50'].first_valid_index():].copy()

        if df_backtest.empty:
            start_date = end_date
            continue


        for name, strategy in strategies.items():
            output = run_and_save_backtest (strategy, df_backtest)
            results.append((start_date, end_date, output, name))

        start_date = end_date

    # Processing and saving results for each strategy
    for strategy_name in strategies.keys():
        strategy_results = [(start, end, output) for start, end, output, name in results if name == strategy_name]
        
        # Filter out results where output is None
        results_with_trades = [(start, end, output) for start, end, output in strategy_results if output is not None and output.get('# Trades', 0) > 1]

        # Sorting by Sharpe Ratio in descending order for top 10 and ascending for bottom 10
        sorted_results = sorted(results_with_trades, key=lambda x: x[2]['Sharpe Ratio'], reverse=True)

        # Get the top 10 results (based on Sharpe Ratio)
        top_10_results = sorted_results[:10]

        # Get the bottom 10 results (based on Sharpe Ratio)
        bottom_10_results = sorted_results[-10:]

        # Calculating metrics for both top and bottom 10
        top_10_metrics = calculate_metrics(top_10_results, periods_per_year, market_return)
        bottom_10_metrics = calculate_metrics(bottom_10_results, periods_per_year, market_return)

        # Saving top 10 and bottom 10 to Excel
        with pd.ExcelWriter(f'{strategy_name}_top_bottom_10_metrics.xlsx') as writer:
            top_10_metrics.to_excel(writer, sheet_name='Top 10', index=False)
            bottom_10_metrics.to_excel(writer, sheet_name='Bottom 10', index=False)

        print(f"Saved top 10 and bottom 10 trades with metrics for {strategy_name} to {strategy_name}_top_bottom_10_metrics.xlsx")


C:\Users\ganes\AppData\Local\Programs\Python\Python312\Lib\site-packages\backtesting\_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

Enter the desired time frame (e.g., '15min', '1hr', '4hr', '8hr', '24hr', '1 week'):  15min


Saved top 10 and bottom 10 trades with metrics for SMA_2_5 to SMA_2_5_top_bottom_10_metrics.xlsx
Saved top 10 and bottom 10 trades with metrics for SMA_5_10 to SMA_5_10_top_bottom_10_metrics.xlsx
Saved top 10 and bottom 10 trades with metrics for SMA_10_15 to SMA_10_15_top_bottom_10_metrics.xlsx
Saved top 10 and bottom 10 trades with metrics for SMA_15_20 to SMA_15_20_top_bottom_10_metrics.xlsx
Saved top 10 and bottom 10 trades with metrics for SMA_20_50 to SMA_20_50_top_bottom_10_metrics.xlsx
Saved top 10 and bottom 10 trades with metrics for EMA_2_5 to EMA_2_5_top_bottom_10_metrics.xlsx
Saved top 10 and bottom 10 trades with metrics for EMA_5_10 to EMA_5_10_top_bottom_10_metrics.xlsx
Saved top 10 and bottom 10 trades with metrics for EMA_10_15 to EMA_10_15_top_bottom_10_metrics.xlsx
Saved top 10 and bottom 10 trades with metrics for EMA_15_20 to EMA_15_20_top_bottom_10_metrics.xlsx
Saved top 10 and bottom 10 trades with metrics for EMA_20_50 to EMA_20_50_top_bottom_10_metrics.xlsx
Sa